In [104]:
import random

import numpy as np
import torch

In [105]:
# chargement des données

fichier = open('villes.txt')
donnees = fichier.read()
villes = donnees.replace('\n', ',').split(',')

# préparation des données

# on rajoute le token . au début et en fin
for ville, i in zip(villes, range(len(villes))):
    villes[i] = '.' + ville + '.'

# création du vocabulaire
vocabulaire = []

for ville in villes:
    for c in ville:
        if c not in vocabulaire:
            vocabulaire.append(c)

vocabulaire = sorted(vocabulaire)
vocabulaire[0] = '.' # 0 est " " et 3 est "." -> on échange
vocabulaire[3] = ' '

# pour convertir char <-> int
char_to_int = {}
int_to_char = {}

for (c, i) in zip(vocabulaire, range(len(vocabulaire))):
    char_to_int[c] = i
    int_to_char[i] = c

# random (0-gram)

In [106]:
''.join(random.choice(vocabulaire) for _ in range(30))

'ednês-ü.ÿçenx-rammêfn.vzôeeœto'

In [107]:
# loss
-torch.log(torch.tensor(1/len(vocabulaire)))

tensor(3.7842)

# unigram

In [108]:
# création du dataset

X = []

for ville in villes:
    for char in ville:
        X.append([char_to_int[char]])

X = torch.asarray(X) # (M, 1)

In [13]:
# modèle uni-gram
P = torch.zeros((len(vocabulaire)))

for i in range(X.shape[0]):
    P[X[i]] += 1

P = P / P.sum(dim=0, keepdim=True)

In [34]:
g = torch.Generator().manual_seed(40+5)

for _ in range(10):
    nom = "."
    while nom[-1] != "." or len(nom) ==1:
        next_char = int_to_char[torch.multinomial(P, num_samples=1, replacement=True, generator=g).item()]
        nom = nom + next_char
    print(nom[1:-1])


a-cqsu
n
o
sayoemnraro

ianèriaesom
aom--ei
l
oitabu


In [12]:
# loss
nll = 0
for i in range(X.shape[0]):
    nll += torch.log(P[X[i, 0]])
-nll/X.shape[0]

tensor(2.9820)

# bigrams

In [109]:
# création du dataset

X = []

for ville in villes:
    for ch1, ch2 in zip(ville, ville[1:]):
        X.append([char_to_int[ch1], char_to_int[ch2]])

X = torch.asarray(X) # (M, 2)

In [110]:
# modèle bigram
P = torch.zeros((len(vocabulaire), len(vocabulaire)))

for i in range(X.shape[0]):
    P[X[i, 0], X[i, 1]] += 1

P = P / P.sum(dim=1, keepdim=True)

In [111]:
100*P[char_to_int['z'], char_to_int['z']]

tensor(0.7436)

In [112]:
g = torch.Generator().manual_seed(42+4)

for _ in range(10):
    nom = "."
    while nom[-1] != "." or len(nom) == 1:
        last_char = nom[-1]
        next_char = int_to_char[torch.multinomial(P[char_to_int[last_char]], num_samples=1, replacement=True, generator=g).item()]
        nom = nom + next_char
    print(nom[1:-1])

layst
stl
pin-s-d-sus
chisspen
moue c-gu-main-peu
llergnch
cheusollet
lle
bllaint-lenévir-sesas-penaiemey
norgitagesa


In [12]:
# loss
nll = 0
for i in range(X.shape[0]):
    nll += torch.log(P[X[i, 0], X[i, 1]])
-nll/X.shape[0]

tensor(2.3906)

# trigrams

In [113]:
# on rajoute le token . au début et en fin
for ville, i in zip(villes, range(len(villes))):
    villes[i] = '.' + ville + "."

In [114]:
# création du dataset

X = []

for ville in villes:
    for ch1, ch2, ch3 in zip(ville, ville[1:], ville[2:]):
        X.append([char_to_int[ch1], char_to_int[ch2], char_to_int[ch3]])

X = torch.asarray(X) # (M, 3)

In [115]:
# modèle trigram
P = torch.zeros((len(vocabulaire), len(vocabulaire), len(vocabulaire)))

for i in range(X.shape[0]):
    P[X[i, 0], X[i, 1], X[i, 2]] += 1

P = P / P.sum(dim=2, keepdim=True)

In [116]:
100*P[char_to_int['z'], char_to_int['z'], char_to_int['z']]

tensor(0.)

In [118]:
g = torch.Generator().manual_seed(4354)

for _ in range(10):
    nom = ".."
    while nom[-1] != "." or len(nom) == 2:
        char_moins_1 = nom[-1]
        char_moins_2 = nom[-2]

        next_char = int_to_char[torch.multinomial(P[char_to_int[char_moins_2], char_to_int[char_moins_1]], num_samples=1, replacement=True, generator=g).item()]
        nom = nom + next_char

    print(nom[2:-1])

igorville
sérômes-ne
pes
mille
carren-d'as
quintotse
print-méneux
lournac
la-venoisille pelle
gace


In [17]:
# loss
nll = 0
for i in range(X.shape[0]):
    nll += torch.log(P[X[i, 0], X[i, 1], X[i, 2]])
-nll/X.shape[0]

tensor(1.8118)